In [2]:
import sys 
import os
sys.path.append(os.path.abspath('../models'))
sys.path.append(os.path.abspath('../dev'))

from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import BasePreprocessor
from mltrainer import Trainer, TrainerSettings, ReportTypes, metrics
from neural_network import NeuralNetwork
from neural_network import DeepNeuralNetwork

from helpers import get_last_accuracy_from_tensorboard

import torch.optim as optim
import torch

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import os

In [ ]:
fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
preprocessor = BasePreprocessor()

batchsize = 64

streamers = fashionfactory.create_datastreamer(batchsize=batchsize, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

trainstreamer = train.stream()
validstreamer = valid.stream()

accuracy = metrics.Accuracy()
loss_func = torch.nn.CrossEntropyLoss()

2025-05-06 17:43:27.064 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/azureuser/.cache/mads_datasets/fashionmnist
2025-05-06 17:43:27.065 | INFO     | mads_datasets.base:download_data:124 - File already exists at /home/azureuser/.cache/mads_datasets/fashionmnist/fashionmnist.pt


In [ ]:
units = [2048, 1024, 512, 256, 128, 64, 32, 16]

epochs = 5
accuracies5 = []
dir = "modellogs/UNITS/EPOCH5/"

for units1 in units:
    row = []
    for units2 in units:
        logdir = f"{dir}u{units1}_u{units2}_e{epochs}"

        settings = TrainerSettings(
            epochs=epochs,
            metrics=[accuracy],
            logdir="modellogs",
            train_steps=100,
            valid_steps=100,
            reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML]
        )

        model = NeuralNetwork(num_classes=10, units1=units1, units2=units2)
        settings.logdir = f"{dir}u{units1}_u{units2}_e{epochs}"

        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_func,
            optimizer=optim.Adam,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau
        )

        trainer.loop()
        
        tb_subdirs = [os.path.join(logdir, d) for d in os.listdir(logdir) if os.path.isdir(os.path.join(logdir, d))]

        # Zoek eerste subdir met tensorboard events
        tb_dir = next((d for d in tb_subdirs if any("events.out.tfevents" in f for f in os.listdir(d))), None)

        if tb_dir:
            last_acc = get_last_accuracy_from_tensorboard(tb_dir)
        else:
            last_acc = np.nan
        
        row.append(last_acc)
    accuracies5.append(row)

In [ ]:
# Plot de heatmap
plt.figure(figsize=(8, 5))
ax = sns.heatmap(
    accuracies5,
    annot=True,
    fmt=".1f",
    xticklabels=units,
    yticklabels=units,
    cmap="YlOrRd",
    cbar_kws={'label': 'Accuracy (%)'}
)
ax.set_xlabel("units2")
ax.set_ylabel("units1")
ax.set_title("Laatste Accuracy per Units-Combinatie epochs = 5")
plt.tight_layout()
plt.savefig("heatmap_epochs5")
plt.show()

In [ ]:
units = [2048, 1024, 512, 256, 128, 64, 32, 16]

epochs = 10
accuracies = []
dir = "modellogs/UNITS/EPOCH10/"

for units1 in units:
    row = []
    for units2 in units:
        logdir = f"{dir}u{units1}_u{units2}_e{epochs}"

        settings = TrainerSettings(
            epochs=epochs,
            metrics=[accuracy],
            logdir="modellogs",
            train_steps=100,
            valid_steps=100,
            reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML]
        )

        model = NeuralNetwork(num_classes=10, units1=units1, units2=units2)
        settings.logdir = f"{dir}u{units1}_u{units2}_e{epochs}"

        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_func,
            optimizer=optim.Adam,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau
        )

        trainer.loop()
        
        tb_subdirs = [os.path.join(logdir, d) for d in os.listdir(logdir) if os.path.isdir(os.path.join(logdir, d))]

        # Zoek eerste subdir met tensorboard events
        tb_dir = next((d for d in tb_subdirs if any("events.out.tfevents" in f for f in os.listdir(d))), None)

        if tb_dir:
            last_acc = get_last_accuracy_from_tensorboard(tb_dir)
        else:
            last_acc = np.nan
        
        row.append(last_acc)
    accuracies.append(row)

In [ ]:
# Plot de heatmap
plt.figure(figsize=(8, 5))
ax = sns.heatmap(
    accuracies,
    annot=True,
    fmt=".1f",
    xticklabels=units,
    yticklabels=units,
    cmap="YlOrRd",
    cbar_kws={'label': 'Accuracy (%)'}
)
ax.set_xlabel("units2")
ax.set_ylabel("units1")
ax.set_title("Laatste Accuracy per Units-Combinatie epochs = 10")
plt.tight_layout()
plt.savefig("heatmap_epochs10")
plt.show()

In [ ]:
units = [512, 256, 128]
epochs = 10
results = []

for units1 in units:
    for units2 in units:

        settings = TrainerSettings(
            epochs=epochs,
            metrics=[accuracy],
            logdir="modellogs",
            train_steps=100,
            valid_steps=100,
            reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML]
        )

        model = NeuralNetwork (num_classes=10, units1=units1, units2=units2)
        settings.logdir = f"modellogs/SGD/u{units1}_u{units2}_e{epochs}"
        
        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_func,
            optimizer=optim.SGD,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau
        )

        trainer.loop()


In [ ]:
units1 = 512
units2 = 256
units3 = 128
epochs = 20

settings = TrainerSettings(
            epochs=epochs,
            metrics=[accuracy],
            logdir="modellogs",
            train_steps=100,
            valid_steps=100,
            reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML]
        )

model = DeepNeuralNetwork (num_classes=10, units1=units1, units2=units2, units3 = units3)
settings.logdir = f"modellogs/DEEP/u{units1}_u{units2}_u{units3}_e{epochs}"
        
trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_func,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau
)

trainer.loop()


In [5]:
batchsize = 64

streamers = fashionfactory.create_datastreamer(batchsize=batchsize, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

trainstreamer = train.stream()
validstreamer = valid.stream()

accuracy = metrics.Accuracy()
loss_func = torch.nn.CrossEntropyLoss()

2025-05-06 17:43:32.126 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/azureuser/.cache/mads_datasets/fashionmnist
2025-05-06 17:43:32.127 | INFO     | mads_datasets.base:download_data:124 - File already exists at /home/azureuser/.cache/mads_datasets/fashionmnist/fashionmnist.pt


In [6]:
units = [1024, 512]
epochs = 10
results = []

settings = TrainerSettings(
    epochs=epochs,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps=100,
    valid_steps=100,
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
    optimizer_kwargs={"lr": 0.0005}
)

model = NeuralNetwork (num_classes=10, units1=units[0], units2=units[1])
settings.logdir = f"modellogs/DEF/u{units[0]}_u{units[1]}_e{epochs}"

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_func,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau
)

trainer.loop()

2025-05-06 17:43:35.467 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs/DEF/u1024_u512_e10/20250506-174335


2025-05-06 17:43:36.534 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 100/100 [00:00<00:00, 102.72it/s]
2025-05-06 17:43:37.848 | INFO     | mltrainer.trainer:report:205 - Epoch 0 train 0.8524 test 0.6396 metric ['0.7736']
100%|██████████| 100/100 [00:01<00:00, 99.41it/s]
2025-05-06 17:43:39.189 | INFO     | mltrainer.trainer:report:205 - Epoch 2 train 0.5516 test 0.5316 metric ['0.8091']
100%|██████████| 100/100 [00:00<00:00, 100.53it/s]
2025-05-06 17:43:40.523 | INFO     | mltrainer.trainer:report:205 - Epoch 4 train 0.4973 test 0.4724 metric ['0.8286']
100%|██████████| 100/100 [00:00<00:00, 102.02it/s]
2025-05-06 17:43:41.837 | INFO     | mltrainer.trainer:report:205 - Epoch 6 train 0.4805 test 0.4657 metric ['0.8291']
100%|██████████| 100/100 [00:00<00:00, 104.73it/s]
2025-05-06 17:43:43.119 | INFO     | mltrainer.trainer:report:205 - Epoch 8 train 0.4449 test 0.4499 metric ['0.8402']
100